In [ ]:
import requests
import json

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url
        self.call_count = 0

    def generate(self, prompt):
        self.call_count += 1
        payload = {
            "model": "llama3.1",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']


from langchain import LLMChain, PromptTemplate

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Define a prompt template for breaking down the problem
breakdown_template = PromptTemplate(
    input_variables=["question"],
    template="Break down the following problem into step-by-step reasoning: {question}"
)

# Define a prompt template for each step
step_template = PromptTemplate(
    input_variables=["step"],
    template="Explain the following step in detail: {step}"
)

# Function to use LlamaLLM with LLMChain
def run_llm_chain(prompt_template, llm, inputs):
    prompt = prompt_template.format(**inputs)
    response = llm.generate(prompt)
    return response

# Define the question
question = "How can I determine the distance traveled by a car if it travels at 50 miles per hour for 4 hours?"

# Run the breakdown chain
breakdown_steps = run_llm_chain(breakdown_template, llm, {"question": question})
print("Breakdown Steps:", breakdown_steps)

# Split the breakdown into individual steps
steps = breakdown_steps.split("\n")

# Run the step chain for each step
for step in steps:
    explanation = run_llm_chain(step_template, llm, {"step": step})
    print("Step Explanation:", explanation)


Breakdown Steps: Step 1: Identify the rate of travel. The car is traveling at 50 miles per hour.
Step 2: Determine the time period over which the car traveled. The car traveled for 4 hours.
Step 3: Multiply the rate by the time to find distance.
Step 4: Calculate the product of 50 and 4, which equals 200 miles.

Therefore, the car traveled 200 miles.
